In [57]:
import numpy as np
import torch
import pandas as pd

def problem(dim):
    dim = dim
    x_solution = torch.randint(low=1, high=9, size=(dim, 1)).type(torch.DoubleTensor)

    # A must be symmetric, therefore we multiply it with its transpose
    A = torch.randint(low=1, high=9, size=(dim, dim)).type(torch.DoubleTensor)
    A = A @ A.T
    b = A @ x_solution

    return x_solution, A, b

In [58]:
# create function
def f(x, A, b):
    return (1/2) * x.T @ A @ x - b.T @ x

def f_grad(x, A, b):
    return A@x - b

def f_hessian(x, A, b):
    return A

In [59]:
def line_search(f, x, p, grad, alpha=1, r=0.8, c=0.9):
  # backtracking algortihm from page 37

    while f(x + alpha * p) > f(x) + (c * alpha * grad(x).T @ p):
        alpha = r * alpha

    return alpha

In [60]:
######################
# Steepest Descent   #
######################

def steepest_descent(A, b):
    initial_vector = torch.randint(low=1, high=9, size=(dim, 1)).type(torch.DoubleTensor)
    
    # set lambda functions
    f_lambda = lambda x: f(x, A, b)
    f_grad_lambda = lambda x: f_grad(x, A, b)

    # set parameters
    stopp = 0.1
    loss = 1000
    i = 0

    # set initial values
    x = initial_vector

    while loss > stopp:
        p = -1 * f_grad_lambda(x)
        alpha = line_search(f_lambda, x, p, f_grad_lambda)

        x_next = x + alpha * p
        loss = torch.norm(f_grad_lambda(x_next), p=2)

        if i % 100 == 0:
            print(f'iteration:[{i}] Loss: {round(loss.item(), 2)}')

        x = x_next
        i+=1
        
    return x

In [61]:
######################
# Newton Method      #
######################

def newton_method(A, b):
    initial_vec = torch.randint(low=1, high=9, size=(dim, 1)).type(torch.DoubleTensor)
    
    # set lambda functions
    f_lambda = lambda x: f(x, A, b)
    f_grad_lambda = lambda x: f_grad(x, A, b)
    f_hessian_lambda = lambda x: f_hessian(x, A, b)

    # set parameters
    stopp = 0.1
    loss = 1000
    i = 0

    # set initial values
    x = initial_vec
    
    while loss > stopp:
        p = -1 * torch.inverse(f_hessian_lambda(x)) @ f_grad_lambda(x)
        alpha = line_search(f_lambda, x, p, f_grad_lambda)

        x_next = x + alpha * p
        loss = torch.norm(f_grad_lambda(x_next), p=2)

        if i % 4 == 0:
            print(f'iteration:[{i}] Loss: {round(loss.item(), 2)}')

        x = x_next
        i+=1
        
    return x

In [62]:
################################
# Quasi-Newton Method (BFGS)   #
################################

def quasi_newton(A,b):
    initial_point = torch.randint(low=1, high=9, size=(dim, 1)).type(torch.DoubleTensor)

    # set lambda functions
    f_lambda = lambda x: f(x, A, b)
    f_grad_lambda = lambda x: f_grad(x, A, b)
    f_hessian_lambda = lambda x: f_hessian(x, A, b)

    # set parameters
    stopp = 0.1
    loss = 1000
    i = 0

    # set initial values
    x = initial_point
    grad = f_grad_lambda(x)
    I = torch.eye(dim).type(torch.DoubleTensor)
    H = torch.clone(I)

    while loss > stopp:
        p = -1 * H @ grad
        alpha = line_search(f_lambda, x, p, f_grad_lambda)

        x_next = x + alpha * p
        grad_next = f_grad_lambda(x_next)

        # BFGS Method 6.1
        s = x_next - x          # 6.5
        y = grad_next - grad    # 6.5
        r = 1 / (y.T @ s)       # 6.14
        H_next = (I - r*s@y.T) @ H @ (I - r*y@s.T) + r*s@s.T   # 6.17

        loss = torch.norm(f_grad_lambda(x_next), p=2)

        if i % 4 == 0:
            print(f'[{i}] Loss: {round(loss.item(), 2)}')

        x = x_next
        grad = grad_next
        H = H_next
        i+=1
        
    return x

In [63]:
################################
# Conjugate Gradient Method    #
################################

def conjugate_gradient(A,b):
    initial_point = torch.randint(low=1, high=9, size=(dim, 1)).type(torch.DoubleTensor)

    # set lambda functions
    f_lambda = lambda x: f(x, A, b)
    f_grad_lambda = lambda x: f_grad(x, A, b)
    f_hessian_lambda = lambda x: f_hessian(x, A, b)

    # set parameters
    stopp = 0.1
    loss = 1000
    i = 0

    # set initial values
    x = initial_point
    r = f_grad_lambda(x)
    p = -r

    while loss > stopp:
        # Conjugate Gradient Method (CG) 5.2
        alpha = (r.T @ r)/(p.T @ A @ p)
        x_next = x + alpha * p

        r_next = r + alpha * A @ p
        beta_next = (r_next.T @ r_next) / (r.T @ r)
        p_next = -r_next + beta_next * p

        loss = torch.norm(f_grad_lambda(x_next), p=2)

        if i % 1 == 0:
            print(f'[{i}] Loss: {round(loss.item(), 2)}')

        x = x_next
        r = r_next
        p = p_next
        i+=1
    return x

# 1) Matrix 10x10:
---

In [64]:
dim = 10
x_solution, A, b = problem(dim)

In [65]:
x1 = steepest_descent(A,b)

iteration:[0] Loss: 5881.34
iteration:[100] Loss: 9.5
iteration:[200] Loss: 4.1
iteration:[300] Loss: 1.82
iteration:[400] Loss: 1.0
iteration:[500] Loss: 0.57
iteration:[600] Loss: 0.4
iteration:[700] Loss: 0.25
iteration:[800] Loss: 0.15


In [66]:
x2 = newton_method(A,b)

iteration:[0] Loss: 864.42
iteration:[4] Loss: 414.66
iteration:[8] Loss: 198.91
iteration:[12] Loss: 95.42
iteration:[16] Loss: 45.77
iteration:[20] Loss: 21.96
iteration:[24] Loss: 10.53
iteration:[28] Loss: 5.05
iteration:[32] Loss: 2.42
iteration:[36] Loss: 1.16
iteration:[40] Loss: 0.56
iteration:[44] Loss: 0.27
iteration:[48] Loss: 0.13


In [67]:
x3 = quasi_newton(A,b)

[0] Loss: 277.46
[4] Loss: 228.35
[8] Loss: 181.57
[12] Loss: 93.61
[16] Loss: 44.84
[20] Loss: 21.51
[24] Loss: 10.32
[28] Loss: 4.95
[32] Loss: 2.37
[36] Loss: 1.14
[40] Loss: 0.55
[44] Loss: 0.26
[48] Loss: 0.13


In [68]:
x4 = conjugate_gradient(A,b)

[0] Loss: 665.17
[1] Loss: 162.08
[2] Loss: 86.5
[3] Loss: 17.44
[4] Loss: 28.25
[5] Loss: 5.61
[6] Loss: 7.44
[7] Loss: 4.87
[8] Loss: 5.41
[9] Loss: 4.8
[10] Loss: 0.0


In [69]:
data = {"solution":x_solution.flatten(),"steepest_descent":x1.flatten(),"newton_method":x2.flatten(),"quasi_newton":x3.flatten(),"conjugate_gradient":x4.flatten()}
pd.DataFrame(data,columns=["solution","steepest_descent","newton_method","quasi_newton","conjugate_gradient"])

,solution,steepest_descent,newton_method,quasi_newton,conjugate_gradient
0,1.0,1.023978,1.000000,1.001900,1.0
1,6.0,6.000552,6.000000,5.998470,6.0
2,6.0,5.971998,5.999658,6.000257,6.0
3,4.0,4.020186,4.000000,4.000968,4.0
4,7.0,6.994415,6.999572,6.998138,7.0
5,1.0,1.011785,1.000342,1.001895,1.0
6,3.0,2.967117,3.000257,2.999623,3.0
7,3.0,2.997116,3.000428,2.999675,3.0
8,2.0,2.011894,2.000428,2.000327,2.0
9,7.0,6.998479,6.999658,6.999373,7.0


# 2) Matrix 20x20:
---

In [70]:
dim = 20
x_solution, A, b = problem(dim)

In [71]:
x1 = steepest_descent(A,b)

iteration:[0] Loss: 2085.24
iteration:[100] Loss: 42.27
iteration:[200] Loss: 14.22
iteration:[300] Loss: 12.94
iteration:[400] Loss: 3.24
iteration:[500] Loss: 3.7
iteration:[600] Loss: 2.86
iteration:[700] Loss: 2.4
iteration:[800] Loss: 6.88
iteration:[900] Loss: 5.34
iteration:[1000] Loss: 4.01
iteration:[1100] Loss: 4.02
iteration:[1200] Loss: 3.06
iteration:[1300] Loss: 2.4
iteration:[1400] Loss: 1.88
iteration:[1500] Loss: 1.4
iteration:[1600] Loss: 1.27
iteration:[1700] Loss: 3.56
iteration:[1800] Loss: 2.77
iteration:[1900] Loss: 2.79
iteration:[2000] Loss: 2.09
iteration:[2100] Loss: 1.24
iteration:[2200] Loss: 1.27
iteration:[2300] Loss: 1.0
iteration:[2400] Loss: 0.79
iteration:[2500] Loss: 0.67
iteration:[2600] Loss: 1.94
iteration:[2700] Loss: 0.59
iteration:[2800] Loss: 1.16
iteration:[2900] Loss: 0.92
iteration:[3000] Loss: 0.91
iteration:[3100] Loss: 0.73
iteration:[3200] Loss: 0.59
iteration:[3300] Loss: 0.47
iteration:[3400] Loss: 0.42
iteration:[3500] Loss: 1.21
ite

In [72]:
x2 = newton_method(A,b)

iteration:[0] Loss: 32709.66
iteration:[4] Loss: 15690.8
iteration:[8] Loss: 7526.86
iteration:[12] Loss: 3610.63
iteration:[16] Loss: 1732.02
iteration:[20] Loss: 830.85
iteration:[24] Loss: 398.56
iteration:[28] Loss: 191.19
iteration:[32] Loss: 91.71
iteration:[36] Loss: 43.99
iteration:[40] Loss: 21.1
iteration:[44] Loss: 10.12
iteration:[48] Loss: 4.86
iteration:[52] Loss: 2.33
iteration:[56] Loss: 1.12
iteration:[60] Loss: 0.54
iteration:[64] Loss: 0.26
iteration:[68] Loss: 0.12


In [73]:
x3 = quasi_newton(A,b)

[0] Loss: 23122.12
[4] Loss: 22539.94
[8] Loss: 19227.8
[12] Loss: 17475.65
[16] Loss: 14721.4
[20] Loss: 11108.36
[24] Loss: 5255.76
[28] Loss: 2521.17
[32] Loss: 1209.39
[36] Loss: 580.13
[40] Loss: 278.28
[44] Loss: 133.48
[48] Loss: 64.02
[52] Loss: 30.7
[56] Loss: 14.72
[60] Loss: 7.05
[64] Loss: 3.38
[68] Loss: 1.61
[72] Loss: 0.77
[76] Loss: 0.36
[80] Loss: 0.17


In [74]:
x4 = conjugate_gradient(A,b)

[0] Loss: 2764.24
[1] Loss: 575.95
[2] Loss: 159.91
[3] Loss: 80.08
[4] Loss: 53.75
[5] Loss: 48.67
[6] Loss: 26.66
[7] Loss: 17.4
[8] Loss: 85.6
[9] Loss: 11.92
[10] Loss: 11.61
[11] Loss: 6.32
[12] Loss: 4.0
[13] Loss: 3.21
[14] Loss: 1.76
[15] Loss: 1.25
[16] Loss: 2.61
[17] Loss: 0.32
[18] Loss: 0.74
[19] Loss: 1.61
[20] Loss: 0.12
[21] Loss: 0.18
[22] Loss: 0.41
[23] Loss: 0.0


In [75]:
data = {"solution":x_solution.flatten(),"steepest_descent":x1.flatten(),"newton_method":x2.flatten(),"quasi_newton":x3.flatten(),"conjugate_gradient":x4.flatten()}
pd.DataFrame(data,columns=["solution","steepest_descent","newton_method","quasi_newton","conjugate_gradient"])

,solution,steepest_descent,newton_method,quasi_newton,conjugate_gradient
0,6.0,6.108826,5.999989,5.999743,6.0
1,6.0,5.909642,6.000004,6.000195,6.0
2,5.0,5.548070,4.999991,4.998771,5.0
3,6.0,5.467225,5.999989,6.001185,6.0
4,4.0,3.697877,3.999998,4.000687,4.0
5,7.0,6.874845,6.999987,7.000272,7.0
6,3.0,3.128218,2.999998,2.999732,3.0
7,7.0,6.985897,6.999989,7.000014,7.0
8,7.0,7.120174,6.999998,6.999731,7.0
9,6.0,5.792840,6.000000,6.000462,6.0


# 3) Matrix 13x13:
---

In [76]:
dim = 13
x_solution, A, b = problem(dim)

In [77]:
x1 = steepest_descent(A,b)

iteration:[0] Loss: 16824.38
iteration:[100] Loss: 12.56
iteration:[200] Loss: 1.41
iteration:[300] Loss: 1.95
iteration:[400] Loss: 1.11
iteration:[500] Loss: 0.66
iteration:[600] Loss: 2.1
iteration:[700] Loss: 1.52
iteration:[800] Loss: 1.11
iteration:[900] Loss: 0.79
iteration:[1000] Loss: 1.05
iteration:[1100] Loss: 0.37
iteration:[1200] Loss: 0.29
iteration:[1300] Loss: 0.82
iteration:[1400] Loss: 0.28
iteration:[1500] Loss: 0.23
iteration:[1600] Loss: 0.25
iteration:[1700] Loss: 0.25
iteration:[1800] Loss: 0.2
iteration:[1900] Loss: 0.15
iteration:[2000] Loss: 0.33
iteration:[2100] Loss: 0.32
iteration:[2200] Loss: 0.36
iteration:[2300] Loss: 0.14
iteration:[2400] Loss: 0.32
iteration:[2500] Loss: 0.28


In [78]:
x2 = newton_method(A,b)

iteration:[0] Loss: 10959.59
iteration:[4] Loss: 5257.3
iteration:[8] Loss: 2521.92
iteration:[12] Loss: 1209.77
iteration:[16] Loss: 580.32
iteration:[20] Loss: 278.38
iteration:[24] Loss: 133.54
iteration:[28] Loss: 64.06
iteration:[32] Loss: 30.73
iteration:[36] Loss: 14.74
iteration:[40] Loss: 7.07
iteration:[44] Loss: 3.39
iteration:[48] Loss: 1.63
iteration:[52] Loss: 0.78
iteration:[56] Loss: 0.37
iteration:[60] Loss: 0.18
iteration:[64] Loss: 0.09


In [79]:
x3 = quasi_newton(A,b)

[0] Loss: 8470.38
[4] Loss: 7773.74
[8] Loss: 6573.2
[12] Loss: 4340.26
[16] Loss: 2033.15
[20] Loss: 970.65
[24] Loss: 465.46
[28] Loss: 223.26
[32] Loss: 107.07
[36] Loss: 51.33
[40] Loss: 24.57
[44] Loss: 11.7
[48] Loss: 5.49
[52] Loss: 2.46
[56] Loss: 0.92
[60] Loss: 0.05


In [80]:
x4 = conjugate_gradient(A,b)

[0] Loss: 1368.23
[1] Loss: 357.5
[2] Loss: 157.64
[3] Loss: 68.48
[4] Loss: 27.99
[5] Loss: 23.74
[6] Loss: 16.88
[7] Loss: 11.0
[8] Loss: 11.0
[9] Loss: 106.28
[10] Loss: 5.24
[11] Loss: 0.21
[12] Loss: 0.17
[13] Loss: 0.02


In [81]:
data = {"solution":x_solution.flatten(),"steepest_descent":x1.flatten(),"newton_method":x2.flatten(),"quasi_newton":x3.flatten(),"conjugate_gradient":x4.flatten()}
pd.DataFrame(data,columns=["solution","steepest_descent","newton_method","quasi_newton","conjugate_gradient"])

,solution,steepest_descent,newton_method,quasi_newton,conjugate_gradient
0,1.0,0.365166,1.000039,1.051843,1.000000
1,4.0,3.907617,3.999980,4.014572,3.999995
2,1.0,1.053752,1.000046,0.996193,0.999996
3,7.0,6.843670,7.000000,7.011216,6.999998
4,3.0,4.071776,3.000026,2.904901,2.999997
5,4.0,4.221355,4.000020,3.979888,3.999997
6,6.0,4.851760,5.999980,6.096681,5.999998
7,1.0,1.283840,1.000026,0.974578,0.999997
8,8.0,8.456674,7.999967,7.962834,8.000001
9,2.0,1.731718,2.000039,2.025203,2.000001


# 4) Matrix 15x15:
---

In [82]:
dim = 15
x_solution, A, b = problem(dim)

In [83]:
x1 = steepest_descent(A,b)

iteration:[0] Loss: 3655.13
iteration:[100] Loss: 20.24
iteration:[200] Loss: 2.94
iteration:[300] Loss: 1.04
iteration:[400] Loss: 0.4


In [84]:
x2 = newton_method(A,b)

iteration:[0] Loss: 4651.85
iteration:[4] Loss: 2231.49
iteration:[8] Loss: 1070.44
iteration:[12] Loss: 513.49
iteration:[16] Loss: 246.32
iteration:[20] Loss: 118.16
iteration:[24] Loss: 56.68
iteration:[28] Loss: 27.19
iteration:[32] Loss: 13.04
iteration:[36] Loss: 6.26
iteration:[40] Loss: 3.0
iteration:[44] Loss: 1.44
iteration:[48] Loss: 0.69
iteration:[52] Loss: 0.33
iteration:[56] Loss: 0.16


In [85]:
x3 = quasi_newton(A,b)

[0] Loss: 8579.1
[4] Loss: 8425.42
[8] Loss: 7050.66
[12] Loss: 5921.98
[16] Loss: 3864.49
[20] Loss: 1811.04
[24] Loss: 868.75
[28] Loss: 416.73
[32] Loss: 199.89
[36] Loss: 95.87
[40] Loss: 45.96
[44] Loss: 22.01
[48] Loss: 10.5
[52] Loss: 4.96
[56] Loss: 2.27
[60] Loss: 0.94
[64] Loss: 0.19


In [86]:
x4 = conjugate_gradient(A,b)

[0] Loss: 798.42
[1] Loss: 413.06
[2] Loss: 325.38
[3] Loss: 113.85
[4] Loss: 46.1
[5] Loss: 37.31
[6] Loss: 13.1
[7] Loss: 12.1
[8] Loss: 9.45
[9] Loss: 18.16
[10] Loss: 6.45
[11] Loss: 3.88
[12] Loss: 2.62
[13] Loss: 0.09


In [87]:
data = {"solution":x_solution.flatten(),"steepest_descent":x1.flatten(),"newton_method":x2.flatten(),"quasi_newton":x3.flatten(),"conjugate_gradient":x4.flatten()}
pd.DataFrame(data,columns=["solution","steepest_descent","newton_method","quasi_newton","conjugate_gradient"])

,solution,steepest_descent,newton_method,quasi_newton,conjugate_gradient
0,5.0,4.972867,5.000016,4.997277,5.004073
1,3.0,2.995273,2.999984,2.999356,2.999964
2,5.0,4.923168,5.000016,4.989368,5.022286
3,6.0,5.943908,5.999918,5.992160,6.015671
4,5.0,4.746962,5.000016,4.967402,5.068848
5,6.0,6.405729,5.999984,6.054502,5.887173
6,3.0,3.042471,3.000066,3.005969,2.986897
7,5.0,5.197907,5.000016,5.027526,4.941291
8,2.0,1.934235,2.000016,1.990350,2.020996
9,7.0,6.959107,6.999918,6.994432,7.014045


# 5) Matrix 17x17:
---

In [88]:
dim = 17
x_solution, A, b = problem(dim)

In [89]:
x1 = steepest_descent(A,b)

iteration:[0] Loss: 1575.77
iteration:[100] Loss: 33.75
iteration:[200] Loss: 12.71
iteration:[300] Loss: 14.91
iteration:[400] Loss: 7.84
iteration:[500] Loss: 4.27
iteration:[600] Loss: 2.76
iteration:[700] Loss: 2.2
iteration:[800] Loss: 1.53
iteration:[900] Loss: 0.96
iteration:[1000] Loss: 0.73
iteration:[1100] Loss: 0.65
iteration:[1200] Loss: 1.49
iteration:[1300] Loss: 0.51
iteration:[1400] Loss: 0.52
iteration:[1500] Loss: 1.29
iteration:[1600] Loss: 0.59
iteration:[1700] Loss: 0.49
iteration:[1800] Loss: 0.42
iteration:[1900] Loss: 0.45
iteration:[2000] Loss: 0.81
iteration:[2100] Loss: 0.41
iteration:[2200] Loss: 0.55
iteration:[2300] Loss: 0.46
iteration:[2400] Loss: 0.4
iteration:[2500] Loss: 0.54
iteration:[2600] Loss: 0.38
iteration:[2700] Loss: 0.63
iteration:[2800] Loss: 0.52
iteration:[2900] Loss: 0.44
iteration:[3000] Loss: 0.37
iteration:[3100] Loss: 0.51
iteration:[3200] Loss: 0.43
iteration:[3300] Loss: 1.08
iteration:[3400] Loss: 0.49
iteration:[3500] Loss: 0.41


In [90]:
x2 = newton_method(A,b)

iteration:[0] Loss: 11045.65
iteration:[4] Loss: 5298.59
iteration:[8] Loss: 2541.73
iteration:[12] Loss: 1219.27
iteration:[16] Loss: 584.88
iteration:[20] Loss: 280.57
iteration:[24] Loss: 134.59
iteration:[28] Loss: 64.56
iteration:[32] Loss: 30.97
iteration:[36] Loss: 14.86
iteration:[40] Loss: 7.13
iteration:[44] Loss: 3.42
iteration:[48] Loss: 1.64
iteration:[52] Loss: 0.79
iteration:[56] Loss: 0.38
iteration:[60] Loss: 0.18
iteration:[64] Loss: 0.09


In [91]:
x3 = quasi_newton(A,b)

[0] Loss: 10462.02
[4] Loss: 10342.58
[8] Loss: 8638.72
[12] Loss: 7616.97
[16] Loss: 5193.01
[20] Loss: 2711.57
[24] Loss: 1300.56
[28] Loss: 623.64
[32] Loss: 298.83
[36] Loss: 142.89
[40] Loss: 67.94
[44] Loss: 31.8
[48] Loss: 14.23
[52] Loss: 5.55
[56] Loss: 0.77


In [92]:
x4 = conjugate_gradient(A,b)

[0] Loss: 1403.87
[1] Loss: 417.19
[2] Loss: 202.58
[3] Loss: 121.89
[4] Loss: 76.27
[5] Loss: 45.86
[6] Loss: 51.79
[7] Loss: 24.49
[8] Loss: 16.1
[9] Loss: 45.2
[10] Loss: 7.67
[11] Loss: 4.45
[12] Loss: 4.14
[13] Loss: 2.68
[14] Loss: 0.56
[15] Loss: 0.38
[16] Loss: 2.66
[17] Loss: 0.07


In [93]:
data = {"solution":x_solution.flatten(),"steepest_descent":x1.flatten(),"newton_method":x2.flatten(),"quasi_newton":x3.flatten(),"conjugate_gradient":x4.flatten()}
pd.DataFrame(data,columns=["solution","steepest_descent","newton_method","quasi_newton","conjugate_gradient"])

,solution,steepest_descent,newton_method,quasi_newton,conjugate_gradient
0,7.0,6.810418,6.999993,6.949821,7.037101
1,1.0,0.852817,1.000046,0.961532,1.015218
2,7.0,6.554739,6.999967,6.883746,7.043488
3,8.0,7.596306,8.000000,7.899319,7.910738
4,7.0,6.925457,6.999961,6.981610,6.978078
5,1.0,1.611392,1.000013,1.160793,0.908806
6,3.0,3.092325,3.000033,3.020696,3.083993
7,8.0,7.914227,8.000000,7.973744,8.113663
8,7.0,6.185307,6.999961,6.794141,6.892527
9,1.0,1.410660,1.000000,1.105900,0.995872
